In [2]:
# import datetime
#
# from trade_ibkr.model import OnExecutionFetchedEvent
# from trade_ibkr.obj import start_app_info
#
# def store_executions():
#     app, _ = start_app_info(is_demo=True)
#
#     async def fetched(e: OnExecutionFetchedEvent):
#         for identifier, df in e.executions.execution_dataframes.items():
#             dest = f"execution-{identifier}.csv"
#             print(f"Saved to: {dest}")
#             df.to_csv(dest)
#
#         app.disconnect()
#
#     app.set_on_executions_fetched(
#         on_execution_fetched=fetched,
#         on_execution_fetch_earliest_time=lambda: datetime.datetime(2022, 2, 10)
#     )
#     app.request_all_executions()

In [8]:
import pandas as pd

def get_executions_df(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    return df.groupby("date").sum()[["realized_pnl", "px_side"]]

get_executions_df("../../execution-461318792.csv")

,realized_pnl,px_side
date,,
2022-02-27,327.10,81.775000
2022-02-28,1254.60,260.098333
2022-03-01,387.46,58.751667
2022-03-02,370.08,61.680000
2022-03-04,-484.50,-42.776667


In [9]:
get_executions_df("../../execution-477836934.csv")


,realized_pnl,px_side
date,,
2022-02-27,-110.2,-44.08
2022-03-01,46.9,18.76
2022-03-02,29.6,11.84
